In [5]:
# Install required packages
# Install required packages
!pip install ninja
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
%cd stylegan2-ada-pytorch # Navigate to the cloned repository
!python setup.py build --force  # Build the custom CUDA kernels for StyleGAN2-ADA
!python setup.py install # Install the repository and its dependencies, including dnnlib

fatal: destination path 'stylegan2-ada-pytorch' already exists and is not an empty directory.
[Errno 2] No such file or directory: 'stylegan2-ada-pytorch # Navigate to the cloned repository'
/content
python3: can't open file '/content/setup.py': [Errno 2] No such file or directory
python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


In [6]:
# Import required libraries
import sys
# Add the path to the stylegan2-ada-pytorch repo
sys.path.append('/content/stylegan2-ada-pytorch') # Append the path to the repository
import dnnlib
import legacy
import torch
import numpy as np
from PIL import Image

import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import gzip
from tqdm.notebook import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")



In [7]:
# Load the pre-trained StyleGAN2-ADA model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"

with dnnlib.util.open_url(model_path) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)  # Load the generator


# !pip install ninja
# !git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
# %cd stylegan2-ada-pytorch

!pip install git+https://github.com/FacePerceiver/facer.git@main

# Install and set up pixel2style2pixel for encoding real images to StyleGAN latent space
!git clone https://github.com/eladrich/pixel2style2pixel.git
%cd pixel2style2pixel
!python setup.py develop
%cd ..


# Download the pretrained pSp model for face encoding
!mkdir -p pretrained_models
!gdown --id 1bMTNWkh5LArlaWSc_wa8VKyq2V42T2z0 -O pretrained_models/psp_ffhq_encode.pt

# Download the pretrained StyleGAN model if needed
!gdown --id 1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT -O pretrained_models/stylegan2-ffhq-config-f.pt

# Install custom CUDA kernels for StyleGAN2
!python setup.py develop



# Add the pSp repository to the path
import sys
sys.path.append("pixel2style2pixel")

# Import pSp modules
from models.psp import pSp
from utils.common import tensor2im


  Cloning https://github.com/FacePerceiver/facer.git (to revision main) to /tmp/pip-req-build-6q_6j_6r
  Running command git clone --filter=blob:none --quiet https://github.com/FacePerceiver/facer.git /tmp/pip-req-build-6q_6j_6r
  Resolved https://github.com/FacePerceiver/facer.git to commit ddd35c76ff840174b8a5403ad1c1255e37b8782b
  Preparing metadata (setup.py) ... done
fatal: destination path 'pixel2style2pixel' already exists and is not an empty directory.
/content/pixel2style2pixel
python3: can't open file '/content/pixel2style2pixel/setup.py': [Errno 2] No such file or directory
/content
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1bMTNWkh5LArlaWSc_wa8VKyq2V42T2z0
From (redirected): https://drive.google.com/uc?id=1bMTNWkh5LArlaWSc_wa8VKyq2V42T2

# Preprocessing

In [16]:

"""# Preprocessing"""

from google.colab import drive
from PIL import Image
from IPython.display import display
import os
import random
import pandas as pd




drive.mount('/content/drive')


def import_image(image_path):
  """Imports an image from the specified path using Pillow library.

  Args:
    image_path: The path to the image file.

  Returns:
    A PIL Image object if successful, otherwise None.
  """
  base_path = "/content/drive/MyDrive/Child Generator/"
  image_path = os.path.join(base_path, image_path)
  try:
    img = Image.open(image_path)
    return img
  except FileNotFoundError:
    print(f"Error: Image file not found at {image_path}")
    return None
  except Exception as e:
    print(f"Error loading image: {e}")
    return None


def get_family(family_id, no_of_images=1):
  """
  Retrieves family information for a given family ID.

  Args:
    family_id: The ID of the family.

  Returns:
    A dictionary containing family information, or None if the family ID is not found.
  """
  try:
    df = pd.read_csv("/content/drive/MyDrive/Child Generator/CSVs/checkpoint3.csv")
    family_data = df[df['family_id'] == family_id].iloc[0]

    father_images = random.sample(family_data['father_images'].split(';'), min(no_of_images, len(family_data['father_images'].split(';'))))
    mother_images = random.sample(family_data['mother_images'].split(';'), min(no_of_images, len(family_data['mother_images'].split(';'))))
    child_images = random.sample(family_data['child_images'].split(';'), min(no_of_images, len(family_data['child_images'].split(';'))))

    family_info = {
        'father_images': father_images,
        'mother_images': mother_images,
        'child_images': child_images,
        'Father_name': family_data['father_name'],
        'Mother_name': family_data['mother_name'],
        'Child_name': family_data['child_name']
    }
    return family_info
  except IndexError:
    print(f"Family with ID {family_id} not found.")
    return None


"""
Sample usage to get a path for say 3 trios
"""

fathers = []
mothers = []
children = []

for i in range(10):
  family = get_family(i, no_of_images=1)
  fathers.append(family.get('father_images'))
  mothers.append(family.get('mother_images'))
  children.append(family.get('child_images'))

print(fathers)
print(mothers)
print(children)

"""
Sample usage to get an image from a path without calling import image
"""

def append_to_base_path(path):
  base_path = "/content/drive/MyDrive/Child Generator/"
  return os.path.join(base_path, path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[['Families/Obama/Barack_Obama/Barack_Obama_20403.jpg'], ['Families/Obama/Barack_Obama/Barack_Obama_20091.jpg'], ['Families/Obama/Barack_Obama_Sr/Barack_Obama_Sr_00002.jpg'], ['Families/Obama/Lolo_Soetoro/Lolo_Soetoro_00001.jpg'], ['Families/Obama/Stanley_Dunham/Stanley_Dunham_00002.jpg'], ['Families/Bush/Jeb_Bush/Jeb_Bush_0030.jpg'], ['Families/Clinton/Bill_Clinton/Bill_Clinton_0029.jpg'], ['Families/Clinton/William_Jefferson_Blythe/William_Jefferson_Blythe_0001.jpg'], ['Families/Reagan/Ronald_Reagan/Ronald_Reagan_0023.jpg'], ['Families/Reagan/Ronald_Reagan/Ronald_Reagan_0012.jpg']]
[['Families/Obama/Michelle_Obama/Michelle_Obama_00007.jpg'], ['Families/Obama/Michelle_Obama/Michelle_Obama_00015.jpg'], ['Families/Obama/Ann_Dunham/Ann_Dunham_00013.jpg'], ['Families/Obama/Ann_Dunham/Ann_Dunham_00005.jpg'], ['Families/Obama/Madelyn_Payne/Madelyn_Payne_00002.jpg'

In [17]:
def load_psp_encoder(device):
    """Load the pretrained pSp encoder model"""
    model_path = 'pretrained_models/psp_ffhq_encode.pt'

    if not os.path.exists(model_path):
        print("Model file not found. Downloading...")
        os.makedirs('pretrained_models', exist_ok=True)
        !gdown --id 1bMTNWkh5LArlaWSc_wa8VKyq2V42T2z0 -O {model_path}

    try:
        ckpt = torch.load(model_path, map_location=device)
        opts = ckpt['opts']

        # Convert opts to namespace and add required attributes
        from argparse import Namespace
        opts = Namespace(**opts)

        # Add missing attributes
        opts.output_size = 1024
        opts.checkpoint_path = model_path
        opts.device = device
        opts.stylegan_weights = 'pretrained_models/stylegan2-ffhq-config-f.pt'
        opts.stylegan_size = 1024
        opts.start_from_latent_avg = True
        opts.learn_in_w = True

        # Ensure StyleGAN weights exist
        if not os.path.exists(opts.stylegan_weights):
            print("StyleGAN weights not found. Downloading...")
            !gdown --id 1EM87UquaoQmk17Q8d5kYIAHqu0dkYqdT -O {opts.stylegan_weights}

        # Create the pSp model
        from models.psp import pSp
        psp_encoder = pSp(opts).to(device).eval()
        print("pSp encoder loaded successfully")
        return psp_encoder

    except Exception as e:
        print(f"Error loading pSp encoder: {str(e)}")
        print("Please ensure you have the pixel2style2pixel repository cloned and set up correctly")
        return None

def get_transforms():
    """Define image transformations for pSp encoder"""
    transforms_dict = {
        'transform': transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
    }
    return transforms_dict

In [21]:
import torchvision.transforms as transforms

def get_latent_vector(img_path, encoder, device):
    """Extract latent vector from image using pSp encoder

    Args:
        img_path: Path to image or PIL Image
        encoder: pSp encoder model (not the generator)
        device: Torch device

    Returns:
        W+ latent vector (1 x 18 x 512)  # StyleGAN2 expects this shape
    """
    if isinstance(img_path, str):
        img = Image.open(img_path).convert('RGB')
    else:
        img = img_path.convert('RGB')

    # Transform image for pSp encoder
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

    img_tensor = transform(img).unsqueeze(0).to(device)

    # Extract latent vector using pSp encoder
    with torch.no_grad():
        # The encoder returns (images, latents)
        _, latents = encoder(img_tensor, randomize_noise=False, return_latents=True)
        # latents should now be shape [1, 18, 512]

    print(f"W+ shape: {latents.shape}")
    return latents


In [22]:
# Average in W+ space with equal weights
def slerp(t, v0, v1):
    """Spherical interpolation between v0 and v1."""
    v0_norm = v0 / torch.norm(v0, dim=-1, keepdim=True)
    v1_norm = v1 / torch.norm(v1, dim=-1, keepdim=True)
    omega = torch.acos((v0_norm * v1_norm).sum(dim=-1, keepdim=True))
    sin_omega = torch.sin(omega)

    return (torch.sin((1 - t) * omega) / sin_omega) * v0 + (torch.sin(t * omega) / sin_omega) * v1

def combine_parents(father_path, mother_path, encoder, generator, device):
    """Combine parent images in W+ latent space

    Args:
        father_path: Path to father image
        mother_path: Path to mother image
        encoder: pSp encoder for extracting latent vectors
        generator: StyleGAN2 generator for generating images from latent vectors
        device: Torch device

    Returns:
        Combined parent image, Combined W+ latent vector
    """
    # Get parent latent vectors using the encoder
    father_latent = get_latent_vector(father_path, encoder, device)
    mother_latent = get_latent_vector(mother_path, encoder, device)

    # Print shapes for debugging
    print(f"Father latent shape: {father_latent.shape}")
    print(f"Mother latent shape: {mother_latent.shape}")

    # Interpolate between parents
    t = 0.5  # 0 = father, 1 = mother
    combined_latent = slerp(t, father_latent, mother_latent)

    # Generate combined image using the generator
    with torch.no_grad():
        # StyleGAN2 expects latents of shape [1, 18, 512]
        combined_image = generator.synthesis(combined_latent, noise_mode='const')

    # Convert to PIL image
    combined_image = combined_image.squeeze().cpu().numpy().transpose(1, 2, 0)
    combined_image = ((combined_image + 1) * 127.5).astype(np.uint8)
    combined_image = Image.fromarray(combined_image)

    return combined_image, combined_latent #TODO make sure we are using this combined latent later instead of reextracting it again

In [23]:
# Load the pSp encoder
psp_encoder = load_psp_encoder(device)

# Main parent combination loop
parent_images = []
parent_latents = []
parent_image_paths = []  # Add this list to store paths
output_dir = "generated_parents"
os.makedirs(output_dir, exist_ok=True)

for i, (father, mother) in enumerate(zip(fathers, mothers)):
    father_path = append_to_base_path(father[0])
    mother_path = append_to_base_path(mother[0])

    # Combine parents using both encoder and generator
    combined_image, combined_latent = combine_parents(
        father_path, mother_path, psp_encoder, G, device
    )

    # Save results
    father_name = os.path.splitext(os.path.basename(father[0]))[0]
    mother_name = os.path.splitext(os.path.basename(mother[0]))[0]
    output_path = os.path.join(output_dir, f"combined_{father_name}_{mother_name}.jpg")
    combined_image.save(output_path)

    # Store results
    parent_images.append(combined_image)
    parent_latents.append(combined_latent)
    parent_image_paths.append(output_path)  # Store the path of the combined image

    # Display first family
    if i == 0:
        plt.figure(figsize=(12, 4))
        plt.subplot(1,3,1)
        plt.imshow(Image.open(father_path))
        plt.title("Father")
        plt.axis("off")

        plt.subplot(1,3,2)
        plt.imshow(Image.open(mother_path))
        plt.title("Mother")
        plt.axis("off")

        plt.subplot(1,3,3)
        plt.imshow(combined_image)
        plt.title("Combined Parents")
        plt.axis("off")

        plt.show()

print(f"Generated {len(parent_image_paths)} combined parent images")

Loading pSp from checkpoint: pretrained_models/psp_ffhq_encode.pt
pSp encoder loaded successfully
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
Father latent shape: torch.Size([1, 18, 512])
Mother latent shape: torch.Size([1, 18, 512])
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
Father latent shape: torch.Size([1, 18, 512])
Mother latent shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
Father latent shape: torch.Size([1, 18, 512])
Mother latent shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
Father latent shape: torch.Size([1, 18, 512])
Mother latent shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
Father latent shape: torch.Size([1, 18, 512])
Mother latent shape: torch.Size

In [24]:
import facer
import torch.nn.functional as F

def analyze_face(image_path):
    """Analyze facial features in an image using facer.

    Args:
        image_path: Path to image or PIL Image

    Returns:
        Tuple of (attribute_map, face_crop_path)
        attribute_map contains normalized scores for facial features
    """
    try:
        # Load and prepare image
        if isinstance(image_path, str):
            try:
                # Convert image to facer format
                image = facer.hwc2bchw(facer.read_hwc(image_path)).to(device=device)
                print(f"Successfully loaded image from {image_path}")
            except Exception as e:
                print(f"Error loading image from {image_path}: {str(e)}")
                return None, None
        else:
            # Convert PIL Image to facer format
            img_path = "temp_face.jpg"
            image_path.save(img_path)
            image = facer.hwc2bchw(facer.read_hwc(img_path)).to(device=device)

        # Initialize face detector and attribute analyzer
        face_detector = facer.face_detector("retinaface/mobilenet", device=device)
        face_attr = facer.face_attr("farl/celeba/224", device=device)

        # Detect faces
        with torch.inference_mode():
            faces = face_detector(image)
            if len(faces["rects"]) == 0:
                print(f"No faces detected in image {image_path}")
                return None, None

            # Get face attributes
            faces = face_attr(image, faces)

            if "attrs" not in faces or len(faces["attrs"]) == 0:
                print(f"Could not extract attributes from face in {image_path}")
                return None, None

            # Get attributes of first face
            face1_attrs = faces["attrs"][0]
            labels = face_attr.labels

            # Create attribute map
            attribute_map = {}
            for prob, label in zip(face1_attrs, labels):
                attribute_map[label] = prob.item()

            print("\nRaw Attribute Map:")
            for label, value in attribute_map.items():
                print(f"{label}: {value:.3f}")

            # Map facer attributes to our required attributes
            final_attribute_map = {
                'Male': attribute_map.get('Male', 0.0),
                'Young': 1 - attribute_map.get('Aged', 0.0),  # Invert 'Aged'
                'Smiling': attribute_map.get('Smiling', 0.0),
                'Eyeglasses': attribute_map.get('Eyeglasses', 0.0),
                'Heavy_Makeup': attribute_map.get('Heavy_Makeup', 0.0),
                'Wearing_Hat': attribute_map.get('Wearing_Hat', 0.0),
                'Oval_Face': attribute_map.get('Oval_Face', 0.0),
                'Pointy_Nose': attribute_map.get('Pointy_Nose', 0.0),
                'Mouth_Slightly_Open': attribute_map.get('Mouth_Slightly_Open', 0.0),
                'Wearing_Earrings': attribute_map.get('Wearing_Earrings', 0.0),
                'Pale_Skin': attribute_map.get('Pale_Skin', 0.0),
                '5_o_Clock_Shadow': attribute_map.get('5_o_Clock_Shadow', 0.0),
                'Chubby': attribute_map.get('Chubby', 0.0),
                'Double_Chin': attribute_map.get('Double_Chin', 0.0),
                'Goatee': attribute_map.get('Goatee', 0.0),
                'Gray_Hair': attribute_map.get('Gray_Hair', 0.0),
                'Mustache': attribute_map.get('Mustache', 0.0),
                'No_Beard': attribute_map.get('No_Beard', 1.0),
                'Receding_Hairline': attribute_map.get('Receding_Hairline', 0.0),
                'Rosy_Cheeks': attribute_map.get('Rosy_Cheeks', 0.0),
                'Straight_Hair': attribute_map.get('Straight_Hair', 0.0),
                'Wavy_Hair': attribute_map.get('Wavy_Hair', 0.0),
                'Wearing_Lipstick': attribute_map.get('Wearing_Lipstick', 0.0)
            }

            print("\nMapped Attribute Values:")
            for label, value in final_attribute_map.items():
                print(f"{label}: {value:.3f}")

            # Save cropped face if path provided
            if isinstance(image_path, str):
                # Get face bounding box
                bbox = faces["rects"][0]  # First face
                x1, y1, x2, y2 = bbox.tolist()

                # Convert back to PIL Image for cropping
                #TODO jean why tf are we cropping faces? Where did this idea come from
                img = Image.open(image_path)
                face_crop = img.crop((x1, y1, x2, y2))
                face_crop = face_crop.resize((256, 256))

                crop_path = image_path.replace('.', '_face.')
                face_crop.save(crop_path)
                print(f"\nSaved cropped face to {crop_path}")
            else:
                crop_path = None

            return final_attribute_map, crop_path

    except Exception as e:
        print(f"Error in analyze_face: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None

# Feature extraction
parent_labels = []
child_labels = []

# Iterate through parent image paths
print("Extracting features from parent images...")
for parent_image_path in parent_image_paths:
    attribute_map, _ = analyze_face(parent_image_path)
    if attribute_map is not None:  # Only add if face was detected
        parent_labels.append((attribute_map, parent_image_path))
    print(f"Processed {parent_image_path}")

print(f"\nExtracting features from child images...")
for child in children:
    child_path = append_to_base_path(child[0])
    attribute_map, _ = analyze_face(child_path)
    if attribute_map is not None:  # Only add if face was detected
        child_labels.append((attribute_map, child_path))
    print(f"Processed {child_path}")

print(f"\nProcessed {len(parent_labels)} parent images and {len(child_labels)} child images")


Extracting features from parent images...
Successfully loaded image from generated_parents/combined_Barack_Obama_20403_Michelle_Obama_00007.jpg


Downloading: "https://github.com/elliottzheng/face-detection/releases/download/0.0.1/mobilenet0.25_Final.pth" to /root/.cache/torch/hub/checkpoints/mobilenet0.25_Final.pth
100%|██████████| 1.71M/1.71M [00:00<00:00, 36.0MB/s]
Downloading: "https://github.com/FacePerceiver/facer/releases/download/models-v1/face_attribute.farl.celeba.pt" to /root/.cache/torch/hub/checkpoints/face_attribute.farl.celeba.pt
100%|██████████| 327M/327M [00:09<00:00, 36.6MB/s]



Raw Attribute Map:
5_o_Clock_Shadow: 0.003
Arched_Eyebrows: 0.191
Attractive: 0.013
Bags_Under_Eyes: 0.539
Bald: 0.006
Bangs: 0.001
Big_Lips: 0.126
Big_Nose: 0.847
Black_Hair: 0.088
Blond_Hair: 0.001
Blurry: 0.023
Brown_Hair: 0.069
Bushy_Eyebrows: 0.926
Chubby: 0.449
Double_Chin: 0.198
Eyeglasses: 0.003
Goatee: 0.001
Gray_Hair: 0.002
Heavy_Makeup: 0.004
High_Cheekbones: 0.953
Male: 0.544
Mouth_Slightly_Open: 0.437
Mustache: 0.001
Narrow_Eyes: 0.565
No_Beard: 0.990
Oval_Face: 0.371
Pale_Skin: 0.000
Pointy_Nose: 0.001
Receding_Hairline: 0.920
Rosy_Cheeks: 0.000
Sideburns: 0.001
Smiling: 0.988
Straight_Hair: 0.071
Wavy_Hair: 0.016
Wearing_Earrings: 0.022
Wearing_Hat: 0.000
Wearing_Lipstick: 0.026
Wearing_Necklace: 0.020
Wearing_Necktie: 0.004
Young: 0.575

Mapped Attribute Values:
Male: 0.544
Young: 1.000
Smiling: 0.988
Eyeglasses: 0.003
Heavy_Makeup: 0.004
Wearing_Hat: 0.000
Oval_Face: 0.371
Pointy_Nose: 0.001
Mouth_Slightly_Open: 0.437
Wearing_Earrings: 0.022
Pale_Skin: 0.000
5_o_Clock

In [25]:

"""# Convert images to vectors"""

# Remove problematic imports and setup
import os
import torch
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm_notebook

# Define device (already defined above)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# We'll use the already loaded StyleGAN2-ADA generator (G) instead of loading a new one
# The generator G is already properly initialized above

# Use the same base path as defined in append_to_base_path function
base_path = "/content/drive/MyDrive/Child Generator/"  # This matches the path used elsewhere in the code
modified_children = [
    [os.path.join(base_path, path[0]) for path in children if path]  # Add [0] to get first element and check if path exists
]

# Flatten the list and ensure each item is a string
modified_children_flat = [item for sublist in modified_children for item in sublist if isinstance(item, str)]
image_paths = parent_image_paths + modified_children_flat  # Combine parent and modified children paths
print(image_paths)

transform = transforms.Compose([
    transforms.Resize((1024, 1024)),  # Resize to 1024x1024
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Generate X_data (latent vectors)
X_data = []
for image_path in tqdm_notebook(image_paths):
    latent_vector = get_latent_vector(image_path, psp_encoder, device)  # Use psp_encoder instead of G
    X_data.append(latent_vector.cpu().numpy())

X_data = np.concatenate(X_data, axis=0)  # Concatenate into a single array


['generated_parents/combined_Barack_Obama_20403_Michelle_Obama_00007.jpg', 'generated_parents/combined_Barack_Obama_20091_Michelle_Obama_00015.jpg', 'generated_parents/combined_Barack_Obama_Sr_00002_Ann_Dunham_00013.jpg', 'generated_parents/combined_Lolo_Soetoro_00001_Ann_Dunham_00005.jpg', 'generated_parents/combined_Stanley_Dunham_00002_Madelyn_Payne_00002.jpg', 'generated_parents/combined_Jeb_Bush_0030_Columba_Bush_0001.jpg', 'generated_parents/combined_Bill_Clinton_0029_Hillary_Clinton_0005.jpg', 'generated_parents/combined_William_Jefferson_Blythe_0001_Virginia_Clinton_Kelley_0004.jpg', 'generated_parents/combined_Ronald_Reagan_0023_Jane_Wyman_0020.jpg', 'generated_parents/combined_Ronald_Reagan_0012_Nancy_Reagan_0014.jpg', '/content/drive/MyDrive/Child Generator/Families/Obama/Malia_Ann_Obama/Malia_Ann_Obama_00013.jpg', '/content/drive/MyDrive/Child Generator/Families/Obama/Natasha_Obama/Natasha_Obama_00019.jpg', '/content/drive/MyDrive/Child Generator/Families/Obama/Barack_Obama

  0%|          | 0/20 [00:00<?, ?it/s]

W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])


In [26]:

"""# Train classifier on each feature"""

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score

# List of features
features = [
    '5_o_Clock_Shadow', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee',
    'Gray_Hair', 'Heavy_Makeup', 'Male', 'Mouth_Slightly_Open',
    'Mustache', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose',
    'Receding_Hairline', 'Rosy_Cheeks', 'Smiling', 'Straight_Hair',
    'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
    'Young'
]

# Features that should be **classified** instead of regressed
binary_features = {'Eyeglasses', 'Heavy_Makeup', 'Smiling', 'Wearing_Hat', 'Male'}

# Extract numerical labels
def extract_labels(data, features):
    """Convert a list of (attribute_dict, image_path) tuples into a NumPy array of labels."""
    return np.array([[sample[0].get(feature, 0) for feature in features] for sample in data])

# Convert parent and child labels to NumPy arrays
parent_labels_np = extract_labels(parent_labels, features)
child_labels_np = extract_labels(child_labels, features)

# Combine data
X_data = np.vstack((parent_labels_np, child_labels_np))  # Shape: (num_samples, num_features)

# Dictionary to store trained models
feature_models = {}

# Train regressors/classifiers for each feature
for i, feature in enumerate(features):
    if i < X_data.shape[1]:  # Ensure feature exists
        y = X_data[:, i]

        # **Binarize binary features**
        if feature in binary_features:
            y = (y >= 0.5).astype(int)  # Convert probability to 0 or 1
            model = RandomForestClassifier(n_estimators=100, random_state=42)
        else:
            model = RandomForestRegressor(n_estimators=100, random_state=42)

        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(X_data, y, test_size=0.2, random_state=42)

        # Train the model
        model.fit(X_train, y_train)

        # Evaluate
        if feature in binary_features:
            train_score = accuracy_score(y_train, model.predict(X_train))
            test_score = accuracy_score(y_test, model.predict(X_test))
            print(f"✔ {feature} classifier trained")
        else:
            train_score = model.score(X_train, y_train)
            test_score = model.score(X_test, y_test)
            print(f"✔ {feature} regressor trained")

        print(f"   ↳ Training Score: {train_score:.3f}")
        print(f"   ↳ Testing Score: {test_score:.3f}\n")

        # Store model
        feature_models[feature] = model
    else:
        print(f"⚠ Warning: No labels found for {feature}, skipping.")

print("\n🎯 All models trained successfully!")

✔ 5_o_Clock_Shadow regressor trained
   ↳ Training Score: 0.947
   ↳ Testing Score: -0.226

✔ Chubby regressor trained
   ↳ Training Score: 0.970
   ↳ Testing Score: 0.816

✔ Double_Chin regressor trained
   ↳ Training Score: 0.970
   ↳ Testing Score: 0.974

✔ Eyeglasses classifier trained
   ↳ Training Score: 1.000
   ↳ Testing Score: 1.000

✔ Goatee regressor trained
   ↳ Training Score: 0.891
   ↳ Testing Score: -0.261

✔ Gray_Hair regressor trained
   ↳ Training Score: 0.842
   ↳ Testing Score: -0.044

✔ Heavy_Makeup classifier trained
   ↳ Training Score: 1.000
   ↳ Testing Score: 1.000

✔ Male classifier trained
   ↳ Training Score: 1.000
   ↳ Testing Score: 1.000

✔ Mouth_Slightly_Open regressor trained
   ↳ Training Score: 0.985
   ↳ Testing Score: 0.716

✔ Mustache regressor trained
   ↳ Training Score: 0.925
   ↳ Testing Score: 0.723

✔ No_Beard regressor trained
   ↳ Training Score: 0.925
   ↳ Testing Score: -0.264

✔ Oval_Face regressor trained
   ↳ Training Score: 0.956
  

In [27]:
"""# Edit parent image (into generated child)
# Compare to real child
# Adjust model
"""

# Replace your existing feature direction functions with these

def get_feature_direction(classifier):
    """Extract feature direction from classifier"""
    if hasattr(classifier, 'coef_'):
        # For linear models like LogisticRegression
        return classifier.coef_[0]
    elif hasattr(classifier, 'feature_importances_'):
        # For tree-based models like RandomForest
        # Project feature importances to latent dimension
        importances = classifier.feature_importances_
        # Expand to match latent dimension by repeating
        expanded = np.repeat(importances, 512 // len(importances) + 1)[:512]
        return expanded
    else:
        raise ValueError("Classifier doesn't have a recognized direction attribute")

def modify_feature(w_vector, direction, strength=1.0):
    """Modify a W+ latent vector along a feature direction

    Args:
        w_vector: The latent vector to modify (1 x num_ws x 512)
        direction: The direction to modify along (512)
        strength: The strength of the modification

    Returns:
        Modified latent vector
    """
    # Make a deep copy to avoid modifying the original
    modified = w_vector.clone()

    # Ensure direction is the right size and type
    direction = torch.tensor(direction, device=modified.device, dtype=modified.dtype)

    # Create a new tensor for the modifications
    modified_tensor = modified.clone()

    # Apply the modification to each layer
    for i in range(modified.size(1)):
        layer = modified[:, i].clone()
        direction_reshaped = direction.view_as(layer)
        modified_tensor[:, i] = layer + direction_reshaped * strength

    return modified_tensor


In [38]:
def train_feature_strengths(parent_pairs, child_images, feature_models, G, device, num_epochs=100):
    """Train feature strength parameters to match child attributes."""
    # Initialize feature strengths with learnable parameters
    feature_strengths = {
        feature: torch.nn.Parameter(torch.tensor(0.5, device=device, requires_grad=True))
        for feature in feature_models.keys()
    }

    # Create optimizer for the strength parameters
    optimizer = torch.optim.Adam(feature_strengths.values(), lr=0.01)

    for epoch in range(num_epochs):
        total_loss = 0.0

        for (father_path, mother_path), child_path in zip(parent_pairs, child_images):
            # Get parent latents
            father_latent = get_latent_vector(father_path, psp_encoder, device)
            mother_latent = get_latent_vector(mother_path, psp_encoder, device)

            # Combine parents
            combined_image, combined_latent = combine_parents(
                father_path, mother_path, psp_encoder, G, device
            )

            # Apply feature modifications with current strengths
            modified_latent = combined_latent.clone()
            for feature, strength in feature_strengths.items():
                direction = get_feature_direction(feature_models[feature])
                modified_latent = modify_feature(modified_latent, direction, strength)

            # Generate modified image
            with torch.no_grad():
                modified_image = G.synthesis(modified_latent)

            # Extract features from modified and target child images
            modified_features = analyze_face(tensor_to_pil(modified_image))[0]
            target_features = analyze_face(child_path)[0]

            # Calculate loss based on feature differences
            feature_losses = []
            for feature in feature_models.keys():
                if modified_features and target_features:  # Check if features were extracted successfully
                    diff = modified_features.get(feature, 0) - target_features.get(feature, 0)
                    feature_losses.append(torch.tensor(diff ** 2, device=device, requires_grad=True))

            if feature_losses:  # Only process if we have valid losses
                loss = torch.stack(feature_losses).mean()

                # Backpropagate and update strengths
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

        # Print progress
        avg_loss = total_loss / len(parent_pairs) if parent_pairs else 0
        print(f"Epoch {epoch}: Loss = {avg_loss:.4f}")

    return {k: v.item() for k, v in feature_strengths.items()}

In [39]:
# Create parent-child pairs and prepare paths
parent_pairs = []
child_paths = []

for i in range(len(fathers)):
    father_path = append_to_base_path(fathers[i][0])
    mother_path = append_to_base_path(mothers[i][0])
    child_path = append_to_base_path(children[i][0])

    parent_pairs.append((father_path, mother_path))
    child_paths.append(child_path)

# Train the feature strengths
print("Training feature strengths...")
optimal_strengths = train_feature_strengths(parent_pairs, child_paths, feature_models, G, device)

# Save the trained strengths
import pickle
with open('optimal_strengths.pkl', 'wb') as f:
    pickle.dump(optimal_strengths, f)

print("Training complete! Strengths saved to 'optimal_strengths.pkl'")

Streaming output truncated to the last 5000 lines.
Attractive: 0.013
Bags_Under_Eyes: 0.671
Bald: 0.002
Bangs: 0.000
Big_Lips: 0.104
Big_Nose: 0.429
Black_Hair: 0.002
Blond_Hair: 0.059
Blurry: 0.005
Brown_Hair: 0.071
Bushy_Eyebrows: 0.224
Chubby: 0.300
Double_Chin: 0.122
Eyeglasses: 0.001
Goatee: 0.001
Gray_Hair: 0.014
Heavy_Makeup: 0.002
High_Cheekbones: 0.891
Male: 0.894
Mouth_Slightly_Open: 0.999
Mustache: 0.002
Narrow_Eyes: 0.970
No_Beard: 0.979
Oval_Face: 0.142
Pale_Skin: 0.002
Pointy_Nose: 0.009
Receding_Hairline: 0.434
Rosy_Cheeks: 0.001
Sideburns: 0.001
Smiling: 0.991
Straight_Hair: 0.097
Wavy_Hair: 0.022
Wearing_Earrings: 0.055
Wearing_Hat: 0.000
Wearing_Lipstick: 0.009
Wearing_Necklace: 0.007
Wearing_Necktie: 0.002
Young: 0.261

Mapped Attribute Values:
Male: 0.894
Young: 1.000
Smiling: 0.991
Eyeglasses: 0.001
Heavy_Makeup: 0.002
Wearing_Hat: 0.000
Oval_Face: 0.142
Pointy_Nose: 0.009
Mouth_Slightly_Open: 0.999
Wearing_Earrings: 0.055
Pale_Skin: 0.002
5_o_Clock_Shadow: 0.013
C

In [46]:

"""# Lil Demo"""

import torch
from PIL import Image
from google.colab import files
import io
import numpy as np
import matplotlib.pyplot as plt

def display_images(images, titles=None, figsize=(15, 5)):
    """Display a list of images in a row with optional titles."""
    num_images = len(images)
    plt.figure(figsize=figsize)

    for i, image in enumerate(images):
        plt.subplot(1, num_images, i + 1)
        plt.imshow(image)
        if titles and i < len(titles):
            plt.title(titles[i])
        plt.axis('off')

    plt.tight_layout()
    plt.show()
def tensor_to_pil(tensor):
    """Convert a tensor image to PIL Image."""
    image = tensor.squeeze().cpu().numpy().transpose(1, 2, 0)
    image = ((image + 1) * 127.5).astype(np.uint8)
    return Image.fromarray(image)

def generate_child(father_img, mother_img, feature_models, encoder, generator, feature_strengths):
    """Generate child image using learned feature strengths"""
    # First combine parents
    combined_image, combined_latent = combine_parents(
        father_img, mother_img, encoder, generator, device
    )

    print("Combined parents successfully")

    # Apply learned feature modifications
    modified_latent = combined_latent.clone()
    for feature, strength in feature_strengths.items():
        direction = get_feature_direction(feature_models[feature])
        modified_latent = modify_feature(modified_latent, direction, strength)

    print("Applied feature modifications")

    # Generate final child image
    with torch.no_grad():
        child_tensor = generator.synthesis(modified_latent)
        child_image = tensor_to_pil(child_tensor)
        print("Generated child image")

    return child_image

def demo_child_generation(feature_strengths):
    """Demo function with added debugging and image saving"""
    print("Upload parent photos...")
    uploaded = files.upload()

    image_paths = list(uploaded.keys())
    if len(image_paths) != 2:
        print("Please upload exactly 2 images")
        return

    father_img = Image.open(io.BytesIO(uploaded[image_paths[0]])).convert('RGB')
    mother_img = Image.open(io.BytesIO(uploaded[image_paths[1]])).convert('RGB')

    print("Generating child image...")
    # Generate child using learned strengths
    child_img = generate_child(
        father_img, mother_img,
        feature_models, psp_encoder, G,
        feature_strengths
    )

    # Save the generated child image
    output_dir = "generated_children"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"generated_child_{time.strftime('%Y%m%d_%H%M%S')}.jpg")
    child_img.save(output_path)
    print(f"Saved generated child image to: {output_path}")

    # Display results
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(father_img)
    plt.title("Father")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(mother_img)
    plt.title("Mother")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(child_img)
    plt.title("Generated Child")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# Add necessary imports
import time
import os
import io
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import files

# Run the demo
with open('optimal_strengths.pkl', 'rb') as f:
    optimal_strengths = pickle.load(f)

demo_child_generation(optimal_strengths)

Upload parent photos...


Saving fathertest.jpg to fathertest (4).jpg
Saving mothertest.jpg to mothertest (4).jpg
Generating child image...
W+ shape: torch.Size([1, 18, 512])
W+ shape: torch.Size([1, 18, 512])
Father latent shape: torch.Size([1, 18, 512])
Mother latent shape: torch.Size([1, 18, 512])
Combined parents successfully
Applied feature modifications
Generated child image
Saved generated child image to: generated_children/generated_child_20250307_181617.jpg
